In [1]:
!pip install transformers datasets trl huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00


In [3]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import evaluate  # To calculate ROUGE metrics

In [51]:
data_path = 'Recommendation.csv'
df = pd.read_csv(data_path)
dataset = Dataset.from_pandas(df)

In [52]:
def process_data_for_dpo(example):
    messages = [{"role": "user", "content": f"Recommend a food list for a person whose gender is {example['Cinsiyet']}, age: {example['Yaş']}, hypertension: {example['Hipertansiyon']}, diabetes: {example['Diyabet']}, give only 5 foods. Respond only the names of recommended foods."},
                {"role": "assistant", "content": ", ".join(eval(example['Recommended Foods']))}]
    return {"messages": messages}

In [53]:

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [54]:

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [55]:
# Veri setlerini işlemden geçirme
processed_train_dataset = train_dataset.map(process_data_for_dpo)
processed_val_dataset = val_dataset.map(process_data_for_dpo)

Map:   0%|          | 0/4944 [00:00<?, ? examples/s]

Map:   0%|          | 0/1236 [00:00<?, ? examples/s]

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=640b4ca5a6254777cc450d4d00b8811599d4671274258d8cdd44e7630f70e8b2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [49]:
# Metric computation
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions[0]

    # Padding tokenlarını düzgün işleme
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id

    # Tahminleri ve referansları decode etme
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # String'leri temizle ve küçük harf standardizasyonu yap
    pred_str = [p.strip().lower().split(", ")[:5] for p in pred_str]  # İlk 5 yemek
    label_str = [l.strip().lower().split(", ")[:5] for l in label_str]  # İlk 5 yemek

    # Accuracy hesaplama: Ground truth yemeklerin kapsanma oranı
    correct_count = 0
    total_count = len(label_str)

    for pred_list, label_list in zip(pred_str, label_str):
        correct_count += len(set(pred_list) & set(label_list)) / len(label_list)

    accuracy = correct_count / total_count if total_count > 0 else 0.0

    return {
        "Accuracy": round(accuracy, 4),
    }

In [50]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Memory optimization: avoids storing unnecessary tensors.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [56]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

model.config.use_cache = False


In [57]:
os.environ["WANDB_DISABLED"] = "true"

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=1,  # Set according to your GPU memory capacity
    gradient_accumulation_steps=4,
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=50,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    # eval_steps=50,  # Frequency of evaluation
    fp16=True,  # Mixed precision
)


trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_val_dataset,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
    #preprocess_logits_for_metrics=preprocess_logits_for_metrics,  # Optimize memory usage
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-57-190604b9e760>:18: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/4944 [00:00<?, ? examples/s]

Map:   0%|          | 0/1236 [00:00<?, ? examples/s]

In [58]:
# Train the model
trainer.train()

# Modeli kaydetme
finetune_name = "SmolLM2-FT-MyDataset"
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,0.754700,0.194732
100,0.163800,0.138092
150,0.139800,0.127493
200,0.127900,0.119021
250,0.117500,0.115328
300,0.114800,0.108437
350,0.103900,0.100711
400,0.099700,0.094907
450,0.097600,0.094194
500,0.088900,0.089682


In [59]:
prompt = "Recommend a food list for a person whose gender is male, age: 53, hypertension: yes, diabetes: yes, give only 5 foods. Respond only the names of recommended foods"
test_prompt = [{"content": prompt, "role": "user"}]

formatted_prompt = tokenizer.apply_chat_template(test_prompt, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

In [66]:
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=100)
output = tokenizer.decode(outputs[0], skip_special_tokens=True)
food = output.split("assistant\n")[1].split(",")[:5]
print(food)

['Drumstick tree', ' Onion rings', ' Focaccia', ' Espresso', ' Pigeon pea']


In [67]:
def compute_accuracy(expected, predicted):
    """
    Gerçek (expected) ve tahmin edilen (predicted) listeler arasındaki eleman bazında eşleşme sayısını hesaplar.
    """
    # Elemanları temizle ve küçük harfe çevirerek normalize et
    expected_set = set(map(str.strip, map(str.lower, eval(expected))))
    predicted_list = list(map(str.strip, map(str.lower, predicted)))

    # Eşleşmeleri say
    match_count = sum(1 for item in predicted_list if item in expected_set)

    # Doğruluk oranı: eşleşen eleman sayısı
    return match_count


In [68]:
from tqdm import tqdm
import random

total_matches = 0  # Toplam eşleşme sayısını tutacak değişken
for person in tqdm(processed_val_dataset, desc="Processing dataset"):
    prompt = f"Recommend a food list for a person whose gender is {person['Cinsiyet']}, age: {person['Yaş']}, hypertension: {person['Hipertansiyon']}, diabetes: {person['Diyabet']}, give only 5 foods. Respond only the names of recommended foods"
    test_prompt = [{"content": prompt, "role": "user"}]

    formatted_prompt = tokenizer.apply_chat_template(test_prompt, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

    outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=100)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    food = output.split("assistant\n")[1].split(",")[:5]
    # Accuracy hesapla
    total_matches += compute_accuracy(person["Recommended Foods"], food)



Processing dataset: 100%|██████████| 1236/1236 [43:31<00:00,  2.11s/it]


In [69]:
ground_truth = 5 * len(processed_val_dataset)

accuracy = total_matches / ground_truth
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 97.90%
